In [1]:
_ = ! pip install web3

In [2]:
import os, sys

from datetime import datetime, timedelta
import time

import requests, json
from web3 import Web3, HTTPProvider

from eth_utils import to_bytes
from eth_keys import keys
from eth_account import Account
from eth_account.messages import SignableMessage

In [17]:
w3 = Web3(HTTPProvider("https://rpc.ankr.com/eth/1e5f9fd3e49ecec581e00815ea1eb41bc459248fd3dcf67716ff3237cc3ab68b"))
# https://1rpc.io/eth

In [18]:
source_token = "0x1f9840a85d5af5bf1d1762f925bdaddc4201f984" # UniSwap
destination_token = "0xdac17f958d2ee523a2206206994597c13d831ec7" # USDT

source_amount = 1000000000000000000
slippage = int(0.05 * 10000) # Convert to bps

In [20]:
eth_wallet = Account.from_key("e5ecbcfc71ad24a2131953ef9a66e4b7ed8ca9a10ffd92104e106fab1f0af4f7")

In [21]:
# 0.  One fresh quote – DO NOT fetch another one before sending
quote  = requests.get(
    "https://api.0x.org/swap/permit2/quote",
    params={
        "chainId": 1,
        "sellToken": source_token,
        "buyToken":  destination_token,
        "sellAmount": source_amount,
        "taker": eth_wallet.address,
        "slippageBps": slippage,
    },
    headers={
        "0x-version" : "v2",
        "0x-api-key" : "50cbb531-4fda-4464-acb4-5e6b5bf0e2d6"
    },
    timeout=10,
).json()

In [23]:
ERC20_ABI: list[dict] = [
    {
        "constant": True,
        "inputs": [{"name": "owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function",
    },
]

erc20 = w3.eth.contract(address=Web3.to_checksum_address(source_token), abi=ERC20_ABI)

sell_amount   = int(quote["sellAmount"])
permit_amount = int(quote["permit2"]["eip712"]["message"]["permitted"]["amount"])
owner_balance = int(erc20.functions.balanceOf(eth_wallet.address).call())

print ("sellAmount        :", sell_amount)
print ("permit.amount     :", permit_amount)
print ("wallet UNI balance:", owner_balance)
print ()
print ("gas fee:", (1.0e-18) * int(quote['transaction']['gas']) * int(quote['transaction']['gasPrice']), "ETH")

sellAmount        : 1000000000000000000
permit.amount     : 1000000000000000000
wallet UNI balance: 57116026654249141437

gas fee: 0.00017165371440994702 ETH


In [24]:
# 1) sign raw 32-byte digest
digest = bytes.fromhex(quote["permit2"]["hash"][2:])
sig_obj = keys.PrivateKey(eth_wallet.key).sign_msg_hash(digest)
sig = sig_obj.to_bytes()
if sig[-1] in (0, 1):  # normalize v
    sig = sig[:-1] + bytes([sig[-1] + 27])

print("python len(sig)       :", len(sig))            # MUST be 65
print("sig[-1] (v)           :", sig[-1])            # 27 or 28

# 2) locate & replace the *exact* placeholder
data = bytes.fromhex(quote["transaction"]["data"][2:])
ZERO97 = b"\x00" * (32 + 65)
off = data.rfind(ZERO97)
print("placeholder offset    :", off)
assert off != -1, "Permit2 placeholder not found"

patched = (
    data[:off]
    + (65).to_bytes(32, "big")
    + sig
    + data[off + len(ZERO97):]
)

# 3) sanity: what will the router see?
len_word = int.from_bytes(patched[off:off+32], "big")
print("len_word @ off        :", len_word)           # MUST be 65
print("len_word hex          :", patched[off:off+32].hex())
print("first 8B of r         :", patched[off+32:off+40].hex())
# how many bytes are left after we injected
print("tail after sig bytes  :", len(patched) - (off + 32 + 65))

assert len(sig) == 65
assert len_word == 65, "length word still wrong"
# ensure we didn’t leave another zero block after ours
assert patched.rfind(ZERO97) < off, "another placeholder still present"

calldata = "0x" + patched.hex()

python len(sig)       : 65
sig[-1] (v)           : 27
placeholder offset    : 1025
len_word @ off        : 65
len_word hex          : 0000000000000000000000000000000000000000000000000000000000000041
first 8B of r         : 5d92491e02437877
tail after sig bytes  : 198


In [25]:
# 4. Build, sign, send
tx: dict = {
    "chainId":  1,
    "to":       Web3.to_checksum_address(quote["transaction"]["to"]),
    "data":     calldata,
    "value":    int(quote["transaction"]["value"]),
    "gas":      int(quote["transaction"]["gas"]),
    "nonce":    w3.eth.get_transaction_count(eth_wallet.address),
}

# Prefer EIP-1559 if present
if ("maxFeePerGas" in quote["transaction"]) and ("maxPriorityFeePerGas" in quote["transaction"]):
    tx["type"] = 2
    tx["maxFeePerGas"] = int(quote["transaction"]["maxFeePerGas"])
    tx["maxPriorityFeePerGas"] = int(quote["transaction"]["maxPriorityFeePerGas"])
else:
    tx["gasPrice"] = int(quote["transaction"]["gasPrice"])

# Pass through optional fields
if "accessList" in quote["transaction"]:
    tx["accessList"] = quote["transaction"]["accessList"]

# Only for eth_call / estimateGas preflight
tx = dict(tx, **{"from": eth_wallet.address})

tx_signed = eth_wallet.sign_transaction(tx)
tx_hash = w3.eth.send_raw_transaction(tx_signed.raw_transaction)

In [26]:
# 5. Check result
tx_hex = w3.to_hex(tx_hash)

print("↗ broadcast :", tx_hex)
print("⏳ waiting …")
print("✅ status   :", w3.eth.wait_for_transaction_receipt(tx_hash).status)  # 1 = success

↗ broadcast : 0xb9ff4819d364e738dd1c3ace38d106a936a5f5b14dab1d2539c0fb4c8e6fe346
⏳ waiting …
✅ status   : 0


In [27]:
print ("https://eth.blockscout.com/tx/" + tx_hex)

https://eth.blockscout.com/tx/0xb9ff4819d364e738dd1c3ace38d106a936a5f5b14dab1d2539c0fb4c8e6fe346
